In [1]:
import os
from typing import List, Dict
from mem0 import Memory
from datetime import datetime

from openai import OpenAI

# Set up environment variables
os.environ["OPENAI_API_KEY"] = "gpustack_b0af60452b6eb71c_e8e6a2a6c779934a140a74749ddfa5fd"  # needed for embedding model
os.environ["OPENAI_BASE_URL"] = "http://localhost/v1"  # needed for embedding model
 

In [2]:
class SupportChatbot:
    def __init__(self):
        # Initialize Mem0 with Anthropic's Claude
        self.config = {
            "llm": {
                "provider": "openai",
                "config": {
                    "model": "qwen3",
                    "temperature": 0.1,
                    "max_tokens": 2000,
                },
            },
            "embedder": {
                "provider": "openai",
                "config": {
                    "model": "qwen3-embedding",
                    "openai_base_url": "http://localhost/v1",
                    "api_key": "gpustack_b0af60452b6eb71c_e8e6a2a6c779934a140a74749ddfa5fd",
                    "embedding_dims": 4096
                },
            },
            "vector_store": {
               "provider": "qdrant",
               "config": { 
                   "collection_name": "memories",
                   "embedding_model_dims": 4096
               }
            }
        }

        self.client = OpenAI(
            api_key=os.environ.get('OPENAI_API_KEY'),
            base_url="http://localhost/v1"
        ) 
        self.memory = Memory.from_config(self.config)

        # Define support context
        self.system_context = """
        You are a helpful customer support agent. Use the following guidelines:
        - Be polite and professional
        - Show empathy for customer issues
        - Reference past interactions when relevant
        - Maintain consistent information across conversations
        - If you're unsure about something, ask for clarification
        - Keep track of open issues and follow-ups
        """

    def store_customer_interaction(self, user_id: str, message: str, response: str, metadata: Dict = None):
        """Store customer interaction in memory."""
        if metadata is None:
            metadata = {}

        # Add timestamp to metadata
        metadata["timestamp"] = datetime.now().isoformat()

        # Format conversation for storage
        conversation = [{"role": "user", "content": message}, {"role": "assistant", "content": response}]

        # Store in Mem0
        self.memory.add(conversation, user_id=user_id, metadata=metadata)

    def get_relevant_history(self, user_id: str, query: str) -> List[Dict]:
        """Retrieve relevant past interactions."""
        return self.memory.search(
            query=query,
            user_id=user_id,
            limit=5,  # Adjust based on needs
        )

    def handle_customer_query(self, user_id: str, query: str) -> str:
        """Process customer query with context from past interactions."""

        # Get relevant past interactions
        relevant_history = self.get_relevant_history(user_id, query)

        # Build context from relevant history
        context = "Previous relevant interactions:\n"
        for memory in relevant_history:
            try:
                context += f"Customer: {memory['memory']}\n"
                context += f"Support: {memory['memory']}\n"
                context += "---\n"
            except Exception as e:
                pass
    
        # Prepare prompt with context and current query
        prompt = f"""
        {self.system_context}

        {context}

        Current customer query: {query}

        Provide a helpful response that takes into account any relevant past interactions.
        """
        print(prompt)
        # Generate response using Claude
        response = self.client.chat.completions.create(
            model="qwen3",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=2000,
            temperature=0.1,
        )

        # Store interaction
        self.store_customer_interaction(
            user_id=user_id, message=query, response=response.choices[0].message.content, metadata={"type": "support_query"}
        ) 
        return response.choices[0].message.content

In [ ]:
chatbot = SupportChatbot()
user_id = "customer_bot"
print("Welcome to Customer Support! Type 'exit' to end the conversation.")

while True:
    # Get user input
    query = input()
    print("Customer:", query)

    # Check if user wants to exit
    if query.lower() == "exit":
        print("Thank you for using our support service. Goodbye!")
        break

    # Handle the query and print the response
    response = chatbot.handle_customer_query(user_id, query)
    print("Support:", response, "\n\n")

Welcome to Customer Support! Type 'exit' to end the conversation.


 你好


Customer: 你好

        
        You are a helpful customer support agent. Use the following guidelines:
        - Be polite and professional
        - Show empathy for customer issues
        - Reference past interactions when relevant
        - Maintain consistent information across conversations
        - If you're unsure about something, ask for clarification
        - Keep track of open issues and follow-ups
        

        Previous relevant interactions:


        Current customer query: 你好

        Provide a helpful response that takes into account any relevant past interactions.
        
Support: 

您好！很高兴为您服务。如果您有任何问题或需要帮助，请随时告诉我，我会尽力为您解答。祝您生活愉快！ 




 我喜欢吃草莓


Customer: 我喜欢吃草莓

        
        You are a helpful customer support agent. Use the following guidelines:
        - Be polite and professional
        - Show empathy for customer issues
        - Reference past interactions when relevant
        - Maintain consistent information across conversations
        - If you're unsure about something, ask for clarification
        - Keep track of open issues and follow-ups
        

        Previous relevant interactions:


        Current customer query: 我喜欢吃草莓

        Provide a helpful response that takes into account any relevant past interactions.
        
Support: 

I'm glad to hear that you like strawberries! If you have any questions or need help with anything related to strawberries, feel free to ask. I'm here to support you! 😊 




...

Customer: 你喜欢什么

        
        You are a helpful customer support agent. Use the following guidelines:
        - Be polite and professional
        - Show empathy for customer issues
        - Reference past interactions when relevant
        - Maintain consistent information across conversations
        - If you're unsure about something, ask for clarification
        - Keep track of open issues and follow-ups
        

        Previous relevant interactions:


        Current customer query: 你喜欢什么

        Provide a helpful response that takes into account any relevant past interactions.
        
Support: 

您好！感谢您的提问，我很乐意为您提供帮助。如果您有任何问题或需要支持，请随时告诉我，我会尽力为您解答。期待与您交流！ 




 1


Customer: 1

        
        You are a helpful customer support agent. Use the following guidelines:
        - Be polite and professional
        - Show empathy for customer issues
        - Reference past interactions when relevant
        - Maintain consistent information across conversations
        - If you're unsure about something, ask for clarification
        - Keep track of open issues and follow-ups
        

        Previous relevant interactions:


        Current customer query: 1

        Provide a helpful response that takes into account any relevant past interactions.
        
Support: 

Hello! I'm here to assist you. Could you please share more details about the issue you're facing? For example, what exactly are you trying to accomplish, or what specific problem you're encountering? I'll do my best to resolve it. Let me know how I can help! 




 我喜欢什么


Customer: 我喜欢什么

        
        You are a helpful customer support agent. Use the following guidelines:
        - Be polite and professional
        - Show empathy for customer issues
        - Reference past interactions when relevant
        - Maintain consistent information across conversations
        - If you're unsure about something, ask for clarification
        - Keep track of open issues and follow-ups
        

        Previous relevant interactions:


        Current customer query: 我喜欢什么

        Provide a helpful response that takes into account any relevant past interactions.
        
Support: 

I'm happy to help! Could you please clarify what you mean by "what" you like? I'd be happy to support you further! 




In [ ]:


client = OpenAI(
    api_key=os.environ.get('OPENAI_API_KEY'),
    base_url="http://localhost/v1"
) 

# Generate response using Claude
response = client.chat.completions.create(
    model="qwen3",
    messages=[{"role": "user", "content": "你好"}],
    max_tokens=2000,
    temperature=0.1,
)

print(response)
